# Na aula anterior...

Qual a principal hipótese do Algoritmo do Vizinho Mais Próximo?

# Hoje!

## KNN

<img src="imagens/knn.png" alt="Drawing" style="width: 250px;"/>

**Para pensar:** Construir um função que calcula a distância entre 2 pontos com n dimensões.

In [ ]:
x = [5, 6, 7]
y = [12, 13, 14]

In [ ]:
s = (5 - 12)**2 + (6 - 13)**2 + (7 - 14)**2
s

147

In [ ]:
s = (x[0] - y[0])**2 + (x[1] - y[1])**2 + (x[2] - y[2])**2
s

147

In [ ]:
x = [5, 6]
y = [12, 13]

In [ ]:
s = (x[0] - y[0])**2 + (x[1] - y[1])**2 + (x[2] - y[2])**2
s

IndexError: list index out of range

In [ ]:
s = 0
for i in range(len(x)):
    s = s + (x[i] - y[i])**2
s

98

In [ ]:
s = (x[0] - y[0])**2 + (x[1] - y[1])**2
s

98

In [ ]:
x = [5, 6, 7]
y = [12, 13, 14]

In [ ]:
s = 0
for i in range(len(x)):
    s = s + (x[i] - y[i])**2
s

147

In [ ]:
s = 0
for i in range(len(x)):
    s = s + (x[i] - y[i])**2

distancia = s**0.5
distancia

12.12435565298214

In [ ]:
def calcula_distancia(p1, p2):
    
    if len(p1) != len(p2):
        raise Exception("O tamanho de p1 não pode ser diferente do tamanho de p2")
    
    s = 0
    for i in range(len(p1)):
        s = s + (p1[i] - p2[i])**2

    distancia = s**0.5
    
    return distancia

In [ ]:
x = [5, 6, 7]
y = [12, 13, 14]
calcula_distancia(x, y)

12.12435565298214

**Para pensar:** Construir um função que calcula a predição da classe de um elemento usando o Vizinho Mais Próximo.

In [ ]:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



def import_drive_file(link, sep):
    """ Cria processo de importação de datasets do Drive para um Data Frame no Colab"""
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    fluff, id = link.split('=')
    print (id) # Verify that you have everything after '='

    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('Filename.csv')  
    df = pd.read_csv('Filename.csv', sep=sep)
    return df

In [ ]:

df = import_drive_file(link='https://drive.google.com/open?id=1q_coLMoKemj7KI8rkcZpylsof0QlnWiZ', sep=",")
df.head()

1q_coLMoKemj7KI8rkcZpylsof0QlnWiZ


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
x = df.iloc[0, :4]
y = df.iloc[5, :4]

calcula_distancia(x, y)

0.6164414002968979

In [ ]:
x = df.iloc[0, :4]
y = df.iloc[55, :4]

calcula_distancia(x, y)

3.416138170507745

Preciso:
    - variável resposta: species
    - variáveis explicativas: sepal_length	sepal_width	petal_length	petal_width
    - dado que eu vou fazer a previsão: ponto 0 (dados_teste)
    - dados para comparação: ponto 1:final (dados_treino)

In [ ]:
conhecidos = df[1:].copy()
desconhecidos = df.loc[0, ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].copy()

In [ ]:
def previcao_knn(d_conhecidos, d_desconhecidos, resposta):

    distancia = []

    for i, elem in d_conhecidos.iterrows():

        distancia.append(calcula_distancia(elem.drop(resposta), d_desconhecidos))

    d_conhecidos['distancia'] = distancia
    
    d_conhecidos.sort_values(by = 'distancia', inplace = True)
    
    previsao = d_conhecidos.head(k)[resposta].iloc[0]
    
    return previsao

In [ ]:
previcao_knn(conhecidos, desconhecidos, 'species')

'setosa'

### Os k Vizinhos Mais Próximos

Podemos extender o algoritmo do Vizinho Mais Próximo ao utilizar uma vizinhança maior, ou seja, os k vizinhos mais próximos. Numa situação de classificação desse tipo, podemos imaginar que cada vizinho votará em uma classe; a classe predominante, então, será a predição do modelo para aquela instância.

<img src="imagens/knn_2.png" alt="Drawing" style="width: 250px;"/>

**Para pensar:** Como extender o seu código para considerar os k Vizinhos Mais Próximos, sendo k um parâmetro a ser escolhido?

In [ ]:
# alterar esse código

def previcao_knn(d_conhecidos, d_desconhecidos, resposta, k):

    distancia = []

    for i, elem in d_conhecidos.iterrows():

        distancia.append(calcula_distancia(elem.drop(resposta), d_desconhecidos))

    d_conhecidos['distancia'] = distancia
    
    d_conhecidos.sort_values(by = 'distancia', inplace = True)
    
    previsao = d_conhecidos.head(1)[resposta].iloc[0]
    
    return previsao

**Para pensar:** Como proceder em casos que a variável resposta é contínua? Alterar vizinho mais próximo para identificar se a variável resposta é contínua ou não e fazer knn classificador ou knn regressor de acordo com isso. Como teste, dado o comprimento de pétala, comprimento de sépala e largura de pétala, encontre a largura da pétala. 

In [ ]:
11/3

3.6666666666666665

### Usando o sklearn

In [ ]:
conhecidos.drop('distancia', axis = 1, inplace = True)

In [ ]:
conhecidos.head()

,sepal_length,sepal_width,petal_length,petal_width,species
17,5.1,3.5,1.4,0.3,setosa
4,5.0,3.6,1.4,0.2,setosa
39,5.1,3.4,1.5,0.2,setosa
27,5.2,3.5,1.5,0.2,setosa
28,5.2,3.4,1.4,0.2,setosa


In [ ]:
var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
var_resp = 'species'

In [ ]:
conhecidos[var_resp]

17        setosa
4         setosa
39        setosa
27        setosa
28        setosa
         ...    
131    virginica
105    virginica
122    virginica
117    virginica
118    virginica
Name: species, Length: 149, dtype: object

In [ ]:
desconhecidos = pd.DataFrame(desconhecidos).T

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
modelo = KNeighborsClassifier()

In [ ]:
modelo.fit(conhecidos[var_expl], conhecidos[var_resp])

KNeighborsClassifier()

In [ ]:
modelo.predict(desconhecidos)

array(['setosa'], dtype=object)

In [ ]:
desconhecidos

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2


**Para pensar:** Como escolher o melhor k?

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
modelo = KNeighborsClassifier(1)
modelo.fit(conhecidos[var_expl], conhecidos[var_resp])
y_pred = modelo.predict(conhecidos[var_expl])
accuracy_score(conhecidos[var_resp], y_pred)

1.0

In [ ]:
modelo = KNeighborsClassifier(5)
modelo.fit(conhecidos[var_expl], conhecidos[var_resp])
y_pred = modelo.predict(conhecidos[var_expl])
accuracy_score(conhecidos[var_resp], y_pred)

0.9664429530201343

In [ ]:
modelo = KNeighborsClassifier(10)
modelo.fit(conhecidos[var_expl], conhecidos[var_resp])
y_pred = modelo.predict(conhecidos[var_expl])
accuracy_score(conhecidos[var_resp], y_pred)

0.9798657718120806

In [ ]:
modelo = KNeighborsClassifier(50)
modelo.fit(conhecidos[var_expl], conhecidos[var_resp])
y_pred = modelo.predict(conhecidos[var_expl])
accuracy_score(conhecidos[var_resp], y_pred)

0.9395973154362416

## Metodologias de Avaliação

Para observar ocorrência de *overfit* e calcular métricas mais confiáveis, podemos utilizar diversos tipos de metodologias, as quais partem do princípio de definir subconjuntos de **treinamento e teste**, separados de forma disjunta. Os dados de treinamento são empregados no ajuste do modelo, enquanto que os exemplos de teste simulam a apresentação de objetos novos, os quais não foram vistos durante o aprendizado.

### Hold Out

Nessa metodologia, o conjunto de dados é dividido em apenas duas partes: treinamento e teste. O primeiro serve para o ajuste do modelo, enquanto o segundo será usado em sua avaliação.

<center><img src="https://s3-sa-east-1.amazonaws.com/lcpi/4fa6e987-2508-400d-a751-a9c29279923d.png" alt="Drawing" style="height: 250px;"/></center>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('dados/iris.csv')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 5)

var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
var_resp = 'species'

x_treino = df_treino[var_expl]
x_teste = df_teste[var_expl]

y_treino = df_treino[var_resp]
y_teste = df_teste[var_resp]

modelo = KNeighborsClassifier(1)
modelo.fit(x_treino, y_treino)

y_teste_pred = modelo.predict(x_teste)
accuracy_score(y_teste, y_teste_pred)

0.9

In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 5)

var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
var_resp = 'species'

x_treino = df_treino[var_expl]
x_teste = df_teste[var_expl]

y_treino = df_treino[var_resp]
y_teste = df_teste[var_resp]

modelo = KNeighborsClassifier(5)
modelo.fit(x_treino, y_treino)

y_teste_pred = modelo.predict(x_teste)
accuracy_score(y_teste, y_teste_pred)

0.9333333333333333

### Amostras

Nessa metodologia, o conjunto de dados também é dividido em duas partes, porém dessa vez esse processo de divisão é realizado N vezes, gerando N modelos, cada um com sua medida de avaliação obitdo em cima do conjunto de testes. Para obter uma medida única, são calculadas a média e o desvio padrão do desempenho avaliado. As amostras podem ser realizadas tanto com reposição quanto sem reposição.

<center><img src="https://s3-sa-east-1.amazonaws.com/lcpi/c82578f4-1372-4c18-8d3a-62be46829805.png" alt="Drawing" style="height: 250px;;"/></center>

In [ ]:
k = 1

metricas = []

for i in range(1000):

    df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = i)

    var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    var_resp = 'species'

    x_treino = df_treino[var_expl]
    x_teste = df_teste[var_expl]

    y_treino = df_treino[var_resp]
    y_teste = df_teste[var_resp]

    modelo = KNeighborsClassifier(k)
    modelo.fit(x_treino, y_treino)

    y_teste_pred = modelo.predict(x_teste)
    metricas.append(accuracy_score(y_teste, y_teste_pred))
    
sum(metricas)/len(metricas)

0.9578333333333299

In [ ]:
k = 5

metricas = []

for i in range(1000):

    df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = i)

    var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    var_resp = 'species'

    x_treino = df_treino[var_expl]
    x_teste = df_teste[var_expl]

    y_treino = df_treino[var_resp]
    y_teste = df_teste[var_resp]

    modelo = KNeighborsClassifier(k)
    modelo.fit(x_treino, y_treino)

    y_teste_pred = modelo.predict(x_teste)
    metricas.append(accuracy_score(y_teste, y_teste_pred))
    
sum(metricas)/len(metricas)

0.9630666666666646

### K-Fold

Nessa metodologia, o conjunto de dados é dividido em K partições, gerando N modelos. Cada modelo será treinado com um conjunto de K-1 partições, tomando a partição restante como teste, na qual serão calculadas as medidas de desempenho. Aqui, cada partição será considerada como teste 1 vez, e como treino k-1 vezes. Da mesma que nas Amostras, aqui são calculadas a édia e o desvio padrão.

<center><img src="https://s3-sa-east-1.amazonaws.com/lcpi/411b2d93-23ea-4cbb-a0d1-a5aafb64fd5b.png" style="height: 250px;;"/></center>

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


,k,acc
12,14,0.973333
6,8,0.973333
7,9,0.973333
13,15,0.973333
11,13,0.966667
15,17,0.966667
9,11,0.966667
1,3,0.966667
21,23,0.960000
19,21,0.960000


## Voltando para o KNN

**Para pensar:** Como escolher o melhor k?

In [ ]:
metricas = {}

kf = KFold(n_splits = 10, shuffle = True, random_state = 1)

    
for k in range(2,30):

  metricas_temp = []
  for i_treino, i_teste in kf.split(df):

      df_treino = df.iloc[i_treino]
      df_teste = df.iloc[i_teste]

      var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
      var_resp = 'species'

      x_treino = df_treino[var_expl]
      x_teste = df_teste[var_expl]

      y_treino = df_treino[var_resp]
      y_teste = df_teste[var_resp]

      modelo = KNeighborsClassifier(k)
      modelo.fit(x_treino, y_treino)

      y_teste_pred = modelo.predict(x_teste)
      metricas_temp.append(accuracy_score(y_teste, y_teste_pred))
  
  metricas[k] = sum(metricas_temp)/len(metricas_temp)


metricas_df = pd.DataFrame(pd.Series(metricas)).reset_index()
metricas_df.columns = ['k', 'acc']
metricas_df.sort_values(by="acc", ascending=False)

**Para pensar:** Qual o efeito no algoritmo se a largura da sépala estiver em centímetros a largura da pétala estiver em milímetros? O que pode ser feito?

**Para pensar:** Qual o efeito da dimensionalidade nesse tipo de algoritmo?

**Possíveis soluções**
- [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
- [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)



In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
metricas = {}

kf = KFold(n_splits = 10, shuffle = True, random_state = 1)
#scaler = StandardScaler()
scaler = MinMaxScaler()

for k in range(2,30):

  metricas_temp = []
  for i_treino, i_teste in kf.split(df):

      df_treino = df.iloc[i_treino]
      df_teste = df.iloc[i_teste]

      var_expl = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
      var_resp = 'species'

      x_treino = df_treino[var_expl]
      x_teste = df_teste[var_expl]

      y_treino = df_treino[var_resp]
      y_teste = df_teste[var_resp]

      x_treino_std = scaler.fit_transform(x_treino)
      x_teste_std = scaler.transform(x_teste)

      modelo = KNeighborsClassifier(k)
      modelo.fit(x_treino, y_treino)

      y_teste_pred = modelo.predict(x_teste)
      metricas_temp.append(accuracy_score(y_teste, y_teste_pred))
  
  metricas[k] = sum(metricas_temp)/len(metricas_temp)


metricas_df = pd.DataFrame(pd.Series(metricas)).reset_index()
metricas_df.columns = ['k', 'acc']
metricas_df.sort_values(by="acc", ascending=False)

,k,acc
12,14,0.973333
6,8,0.973333
7,9,0.973333
13,15,0.973333
11,13,0.966667
15,17,0.966667
9,11,0.966667
1,3,0.966667
21,23,0.960000
19,21,0.960000


**Para pensar:** É possível utilizar dados qualitativos no algoritmo do Vizinho Mais Próximo? No exemplo anterior, adicione a informação da espécie como variável explicativa. O modelo melhorou?